In [5]:
import numpy as np
import os
import utils
from grid import Grid
from gs import groundstate
from exc import excitations
from plotting import plot2D, plot_cns
import matplotlib.pyplot as plt


def main():
    args = utils.parse_args()

    output_dir = utils.create_output_dir()

    config = utils.read_config(args.config)

    Lx = config["grid"]["Lx"]
    Ly = config["grid"]["Ly"]
    max_iter = config["max_iter_psi"]
    dJUs = np.arange(**config["physics"]["dJU"])
    N = config["grid"]["N"]
    muU = eval(config["physics"]["Mu"])

    grid = Grid(Lx, Ly)

    omega0s = np.zeros(len(dJUs))
    omega1s = np.zeros(len(dJUs))
    omega2s = np.zeros(len(dJUs))

    for count in range(len(dJUs)):
        dJU = dJUs[count]
        gs = groundstate(max_iter, N, dJU, muU)
        cns = gs.cns()
        # print("cns = ", cns)
        psi0 = gs.psi0(cns)
        # print("psi0 = ", psi0)
        exc = excitations(grid, gs, cns)
        uks, vks, omegaklambda = exc.calculate_matrices()
        # print("omegaklambda = ", omegaklambda)
        omega0s[count] = omegaklambda[1][5][5]
        omega1s[count] = omegaklambda[2][5][5]
        omega2s[count] = omegaklambda[3][5][5]
    plt.plot(dJUs, omega0s, label=r"$\omega_0$")
    plt.plot(dJUs, omega1s, label=r"$\omega_1$")
    plt.plot(dJUs, omega2s, label=r"$\omega_2$")
    plt.legend()
    plt.xlabel(r"$\Delta J/U$")
    plt.ylabel(r"$\omega$")
    plt.show()

    # omega0U = gs.omega0U(cns)
    
    # n0 = gs.n0(cns)

    # plt.plot(cns)
    # plt.xlim(0, 70)
    # plt.show()
    # psi0 = psi0(cns, N)
    # n0 = n0(cns, N)
    # omega0U = omega0(cns, N, dJU, muU, psi0)

    # n0s[count] = n0
    # omega0U = - 2 * dJU * psi0 * psi0 + omega0U
    # for dJU in dJUs:
    #     cns = psi(config["max_iter_psi"], config["grid"]["N"], 0.5, eval(config["physics"]["Mu"]))
    #     cnslist.append(cns)

    # plot_cns(cnslist, r'$n$', r'$\bar{c}_n$', os.path.join(output_dir, "csn.png") )


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/ragheed/Library/Jupyter/runtime/kernel-v2-20743XxL2sA1gKdj4.json


SystemExit: 2